# Homework11

Exercises with text processing and NLP modeling

## Goals

- Understand similarities and differences between the processes of working with text, images and tabular data
- Practice with different methods of encoding and modeling text data
- See different methods for extracting information or patterns from text datasets

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework.

In [ ]:
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/text_utils.py

In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from data_utils import display_silhouette_plots, object_from_json_url
from text_utils import get_top_words

You can tell it's gonna be a good homework from the number of imports.
# 🙃

## Have protein, need seasoning

Let's create a model to help us season our foods. In the end, what we want is a model that receives a short list of ingredients and returns a list of seasonings or complementary ingredients for our original ingredients list.

In order to do that we need a dataset of recipes. We'll load that into a text dataset where each recipe is a document and the ingredients are our document *tokens*.

Let's take a look at the recipe dataset and become familiar with the data and how it's organized.

We'll load our recipes and do a bit of exploratory data analysis to look for patterns first to see if this kind of modeling makes any sense.

### Load Data

Here's our dataset. Let's load it into an object for inspection:

In [ ]:
DATA_PATH = "https://raw.githubusercontent.com/PSAM-5020-2025F-A/5020-utils/refs/heads/main/datasets/text/recipes/recipes_min16.json"
recipes_obj = object_from_json_url(DATA_PATH)

### Look at Data

How's the data organized?

How many recipes do we have?

Do all recipes have the same number of ingredients?

Anything else stand out about the data?

In [ ]:
# TODO: Look at Data here
recipes_obj[:3]    # shows first few recipes
# TODO: How many recipes
num_recipes = len(recipes_obj)
num_recipes
# TODO: How many ingredients do the shortest and longest recipes have?
ingredient_lengths = [len(r["ingredients"]) for r in recipes_obj]
min_len = min(ingredient_lengths)
max_len = max(ingredient_lengths)

### Create Input Features

Our dataset doesn't really have to be a `DataFrame` here. It can, but it doesn't have to be.

Each recipe right now is described as a list of ingredients, but what we really want is a list of *sentences*, where each *sentence* is a Python `string` with all of the ingredients for a given recipe.

Instead of:<br>```["salt", "baking soda", "water", "mushroom"]```,

we want:<br>```"salt baking soda water mushroom"```

The `join()` function might help.

Another thing to consider is wether we want to do anything special about multi-word ingredients, like *baking soda*.

Do we want to let our vectorizer (spoiler) split that into two tokens, or do we want to guarantee that *baking* and *soda* always stay together? 

In [ ]:
# TODO: turn list of objects into list of strings
recipe_texts = [" ".join(recipe["ingredients"]) for recipe in recipes_obj]

# Inspect first few results
recipe_texts[:5]

### Encode Data

The fun part.

Let's vectorize our list of ingredient strings into a sparse document matrix using `CountVectorizer` or `TfidfVectorizer`.

The resulting matrix will have one row for each recipe, and the columns will encode the ingredients.

In [ ]:
# TODO: Vectorize ingredients from our recipe list
vectorizer = TfidfVectorizer()   # you could also use CountVectorizer()
X = vectorizer.fit_transform(recipe_texts)

# TODO: How many words are in our vocabulary?
vocab_size = len(vectorizer.vocabulary_)
vocab_size

### Cluster Data

Now that we have our recipes/documents vectorized we can study them a little bit, and look for patterns.

What happens if we cluster our recipes ? What do the cluster centers represent ?

When might this be useful ?

In [ ]:
# TODO: cluster recipes

# number of clusters
k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

# Cluster assignments for each recipe
recipe_clusters = kmeans.labels_

recipe_clusters[:20]

### Cluster Centers

Use the `get_top_words()` function to decode the `cluster_centers` back into ingredients.

In [ ]:
# TODO: Look at cluster centers

centers = kmeans.cluster_centers_
feature_names = vectorizer.get_feature_names_out()

for i in range(centers.shape[0]):
    print(f"\n=== Cluster {i} ===")
    
    center_vec = centers[i].reshape(1, -1)
    
    top_words = get_top_words(center_vec, feature_names, 10)
    print(top_words)


### Interpretation

<span style="color:hotpink">
What do these cluster centers represent ?<br>
Is there anything interesting about recipe cluster centers ?<br>
</span>

<span style="color:hotpink;">
Each cluster center shows the “typical ingredients” for that group of recipes. 
The top ingredients tell us what kind of recipes are in each cluster:

- Cluster 0: lime, cilantro, chili → Mexican or Latin dishes
- Cluster 1: cheese, basil, garlic → Italian dishes like pasta or pizza
- Cluster 2: soy sauce, sesame, ginger → Asian stir-fry recipes
- Cluster 3: olive oil, tomatoes, parsley → Mediterranean dishes
- Cluster 4: flour, sugar, butter, vanilla → baked goods or desserts
- Cluster 5: cheese, tortilla, salsa → Mexican/Tex-Mex dishes

Interesting things:

- Some common ingredients (like garlic, salt, pepper) appear in many clusters.  
- Clustering shows that recipes with similar ingredients naturally group together, even without looking at instructions.
</span>

### Plot Clusters

Let's plot our clusters to see if we have to adjust any of the clustering parameters.

Since we can't plot in $500$ dimensions, we should use `TruncatedSVD` to look at our clusters in $2D$ and $3D$.

In [ ]:
# TODO: TruncatedSVD to reduce the dimensions of our feature space
svd = TruncatedSVD(n_components=2, random_state=42)
X_reduced = svd.fit_transform(X)

# TODO: plot clusters
plt.figure(figsize=(8,6))

colors = cm.get_cmap('tab10', k)  # k = number of clusters
for i in range(k):
    plt.scatter(
        X_reduced[recipe_clusters==i, 0], 
        X_reduced[recipe_clusters==i, 1], 
        label=f'Cluster {i}', 
        alpha=0.7
    )

plt.xlabel("SVD Component 1")
plt.ylabel("SVD Component 2")
plt.title("Recipe Clusters in 2D")
plt.legend()
plt.show()

### Interpretation

<span style="color:hotpink">
What does the graph look like ?<br>
Are the clusters well-separated ?
</span>

<span style="color:hotpink;">
The plot shows all recipes in 2D after simplifying their ingredient lists using TruncatedSVD.
Each color is a different cluster.

Observations:

Cluster 4 (purple) is quite separate from the others, so these recipes are likely very different (maybe desserts or baked goods).

The other clusters (0, 1, 2, 3, 5) overlap a lot, which means these recipes share many ingredients and aren’t clearly separated in 2D.

Some clusters stand out more than others, but many recipes have ingredients in common, making it hard to fully separate them.

Conclusion:

The clustering groups recipes in a meaningful way, but since ingredients overlap, the clusters aren’t completely distinct.

Using more clusters or more dimensions might make the separation clearer.
</span>

### Plot Silhouette Plots

We can also check the quality of our clustering by looking at the silhouette plots that we get from calling:<br>
`display_silhouette_plots(vectors, clusters)`.

In [ ]:
display_silhouette_plots(X, recipe_clusters)

### Interpretation

<span style="color:hotpink">
How many clusters did you end up with ?<br>
How do they look ?<br>
</span>

<span style="color:hotpink;">
I used 6 clusters total.

Clusters 4 and 5 look the strongest because their silhouette scores are mostly high. This means the recipes in those groups are very similar to each other.

The other clusters (0, 1, 2, 3) have lower scores and more variation, so those groups are not as cleanly separated.

A few recipes go slightly below 0, which means they might fit better in another cluster.

Overall:
The clustering works, but only some groups are very clear. Many recipes share similar ingredients, so the separation isn’t perfect.
</span>

## Recipe Completion

Ok. On to the main event.

Let's create some recipes.

We'll do this using a technique similar to what is used for movie/product recommendations. Given an initial set of ingredients, we'll look at recipes that have similar ingredients and "recommend" additional ingredients.

We already have all of the recipes in our dataset encoded as `tf-idf` vectors. The rest of our algorithm will be something like:
1. Start with an initial set of ingredients
2. Encode ingredients
3. Find a set of recipes that are similar to our list of ingredients
4. Find common ingredients that are in the similar recipes, but not in our list of ingredients
5. Pick representative ingredient to add to recipe
6. Repeat

Let's start.

### 1. Initial list of ingredients

This is just a string with ingredients:

In [ ]:
recipe_seed_str = "tofu" # feel free to change this

### 2. Encode ingredients

Transform the string into a `tf-idf` vector:

In [ ]:
# TODO: transform string into sparse vector

recipe_seed_vct = vectorizer.transform([recipe_seed_str])
recipe_seed_vct


### 3. Find similar recipes

The meat of the algorithm. No pun intended.

In order to find similar recipes, we'll first calculate the distance between our current list of ingredients and all recipes in our dataset.

We can start with euclidean distance and later try other kinds, but the overall processing will be the same:

1. Start with an empty list to store distances
2. Loop over the `tf-idf` recipe vectors and for each vector:
   1. Subtract the ingredient list
   2. Square the difference (to square a sparse matrix `A`, use `A.multiply(A)`)
   3. Sum the terms of the result
   4. Take the square root of the sum
   5. Append to distance list
3. Find the indices of the smallest distances (this operation is called `argsort` and will give us the indices of the recipes that are most similar to our list of ingredients)
4. Check the recipes to see if they are indeed similar (`inverse_transform()` the vectors at the indices calculated above)

In [ ]:
# argsort a list (get sequence of indices that would sort the list)
# https://stackoverflow.com/a/3382369
def argsort(L, reverse=False):
  return sorted(range(len(L)), key=L.__getitem__, reverse=reverse)

In [ ]:
import numpy as np


# TODO: list to keep distances
recipe_dists = []

# TODO: loop over vectors and append euclidean distances to list
for v in X:                    # X = all recipe tf-idf vectors
    diff = v - recipe_seed_vct
    sq = diff.multiply(diff)  # elementwise square (sparse-safe)
    dist = np.sqrt(sq.sum())  # Euclidean distance
    recipe_dists.append(dist)
    
# TODO: argsort list of distances to find indices of similar recipes
similar_idxs = argsort(recipe_dists)

# TODO: check first 4 recipes
for i in similar_idxs[:4]:
    print(vectorizer.inverse_transform(X[i])[0])

### 4. Find ingredients to recommend

We have a way to get a set of similar recipes with similar ingredients, and now want to find a *meaningful*, or *representative*, ingredient to add to our ingredients list.

Let's consider ingredients in the $16$ most similar recipes. What we are trying to do is find an ingredient that is in a lot of these recipes, but not yet in our list of ingredients.

There are many possible ways of doing this. We could count the number of times different ingredients show up in these $16$ recipes using Python dictionaries and/or sets, but what we're trying to do here is very similar to what a `TfidfVectorizer` does: calculate relative importance of terms in a series of documents.

Let's re-encode these $16$ recipes using their own separate `TfidfVectorizer`, then sum the importance of each ingredient and look at ingredients with the highest importance scores.

We could re-use the vectors/scores from the original `TfidfVectorizer`, but they're gonna be influenced by the relative frequencies of all of the ingredients that showed up in all of the recipes. Using a separate vectorizer is a little bit more precise.

The steps we need to take are:

1. Separate the $16$ recipes most similar to our list of ingredients
   1. We have lots of representations of our recipes, but `recipes` (list of strings) might be the easiest one to use here
2. Create a new `TfidfVectorizer` and encode the $16$ recipes
3. Sum the resulting vectors to get overall importance scores for each ingredient/token
4. Convert resulting vector to a list using `A.tolist()[0]`
5. `argsort` the importance scores to get sequence of ingredient indices ordered from most to least important
6. Find the most important ingredient that isn't on the ingredient list

In [ ]:
# Imports
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from data_utils import object_from_json_url

# 1. Load recipes dataset
DATA_PATH = "https://raw.githubusercontent.com/PSAM-5020-2025F-A/5020-utils/refs/heads/main/datasets/text/recipes/recipes_min16.json"
recipes_obj = object_from_json_url(DATA_PATH)

# 2. Convert each recipe's list of ingredients into a single string
recipe_texts = [" ".join(recipe["ingredients"]) for recipe in recipes_obj]

# 3. Convert all recipes to TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(recipe_texts)

# 4. Seed ingredient
recipe_seed_str = "tofu"  # change this to try a different ingredient

# 5. Transform seed into tf-idf vector
recipe_seed_vct = vectorizer.transform([recipe_seed_str])

# 6. Compute Euclidean distances to all recipes
recipe_dists = []
for v in X:  # X = tf-idf vectors of all recipes
    diff = v - recipe_seed_vct
    sq = diff.multiply(diff)
    dist = np.sqrt(sq.sum())
    recipe_dists.append(dist)

# 7. Define argsort function
def argsort(L, reverse=False):
    return sorted(range(len(L)), key=L.__getitem__, reverse=reverse)

# 8. Get indices of recipes most similar to the seed
similar_idxs = argsort(recipe_dists)

# TODO: Get 16 most similar recipes
k = 16
top_recipes = [recipe_texts[i] for i in similar_idxs[:k]]

# TODO: Encode the 16 recipes
local_vectorizer = TfidfVectorizer()
local_vcts = local_vectorizer.fit_transform(top_recipes)

# TODO: Sum the recipe vectors by column to get ingredient importance scores
ingredient_scores = local_vcts.sum(axis=0).tolist()[0]

# TODO: argsort the importance scores
sorted_idx = argsort(ingredient_scores, reverse=True)

# TODO: Find most important ingredient not yet on the list of ingredients
current_ingredients = set(recipe_seed_str.split())
for idx in sorted_idx:
    ing = local_vectorizer.get_feature_names_out()[idx]
    if ing not in current_ingredients:
        print("Recommended ingredient:", ing)
        break


### 5. Add ingredient to recipe

This is simply adding a word to `recipe_seed_str`

In [ ]:
# TODO: add the first important ingredient to list of ingredients
# Add the recommended ingredient to the seed list
recipe_seed_str += " " + ing


### 6. Repeat (Optional)

Now we can repeat this process until we get an empty list of important ingredients: 
1. Encode current recipe
2. Find similar recipes
3. Find important ingredients
4. Add important ingredient

Might be helpful to define a couple of functions, like `find_similar_recipes()` and `find_important_ingredients()`...

Only do this step if you're really curious about experimenting with generating unconventional ingredient lists. It's not going to be graded.

In [ ]:
# TODO: Create find_similar_recipes(ingredients, recipes, vectorizer)

# TODO: Create find_important_ingredients(recipes)

# TODO: Create recipe by repeating calls to find_similar_recipes() and find_important_ingredients()